In [2]:
import numpy as np
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

from src.datasets.mnist import numpy_to_image
from src.datasets.utils import get_dataset
from src.estimator.OLS import OLSClient
from src.server import Server

cmap = LinearSegmentedColormap.from_list("black_white", [(0, 'blue'), (1, 'red')])

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
(X_train, y_train), (X_test, y_test) = get_dataset("mnist")
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000, 10), (10000, 784), (10000, 10))

In [5]:
c = OLSClient(-1, X_train, y_train, lambda x: 1 / (1 + x))

In [6]:
est = c.local_estimate()

In [7]:
est.shape

(784, 10)

In [8]:
pred = X_test @ est

https://machinelearningmastery.com/polynomial-features-transforms-for-machine-learning/
https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mnist_filters.html
https://jasondeden.medium.com/expand-your-horizons-d72f15e7ffa
https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.22821&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false

In [ ]:
((pred - y_train) ** 2).sum()

In [ ]:
#X, y = X[np.logical_or(y == 1, y == 7)],  y[np.logical_or(y == 1, y == 7)]

In [ ]:
N = 5
clients = []

num_samples = np.random.randint(10, 100, N)

for i in range(N):
    idx = np.random.choice(range(X.shape[0]), num_samples[i])
    X_i, y_i = X[idx], y[idx]
    clients.append(OLSClient(i, X_i, y_i, lambda x: 1 / (1 + x)))

server = Server(clients)

In [ ]:
server.aggregate(unbiased=True)

In [ ]:
arr = clients[2].local_estimate()
arr = (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
arr.reshape((28, 28), order="C") * 255

In [ ]:
clients[2].local_estimate().reshape((28, 28)) * 1e20

In [ ]:
sns.heatmap(clients[2].local_estimate().reshape((28, 28), order="C"), cmap=cmap)

In [ ]:
sns.heatmap(server.mtl[4].reshape((28, 28), order="C"), cmap=cmap)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./src/datasets/data/mnist/root', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Define a linear regression model
class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Initialize the model
input_size = 28 * 28  # Input size is the flattened image size
output_size = 10  # Output size for linear regression
model = LinearRegression(input_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.view(inputs.size(0), -1)  # Flatten the input images
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss
        labels = torch.eye(10)[labels]
        loss = criterion(outputs, labels.float())  # Using MSE loss for regression

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:  # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')


In [ ]:
model.linear.weight.data[0].shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract the weights from the linear layer of the model
weights = model.linear.weight.data[9].numpy().reshape(28, 28)  # Assuming input_size is 28 * 28

# Plot the weights as an image
plt.figure(figsize=(8, 6))
plt.imshow(weights, cmap='gray')
plt.colorbar()
plt.title('Weights of Linear Regression Model')
plt.xlabel('Pixel Column')
plt.ylabel('Pixel Row')
plt.show()
